In [1]:
from pyhmmer.plan7 import HMMFile
from pyhmmer.easel import SequenceFile
from pyhmmer import hmmsearch
import sys, os, glob, gzip, warnings
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

import soothsayer as sy

ModuleNotFoundError: No module named 'pyhmmer'

In [17]:
# with HMMFile("/Users/jolespin/Databases/KOFAM/test.hmm") as hmm_file:
class Dummy():pass
opts = Dummy()

opts.proteins = "./test.faa"
opts.kofam_database = "/Users/jolespin/Databases/KOFAM"
opts.evalue = 0.1
opts.n_jobs = 0

# df_meta_kofam = pd.read_csv(os.path.join(opts.kofam_database, "ko_list"), sep="\t", index_col=0)
# ko_to_data = df_meta_kofam.T.to_dict()


ko_to_data = defaultdict(dict)
with open(os.path.join(opts.kofam_database, "ko_list"), "r") as f:
    header = next(f).strip().split("\t")[1:]
    for line in f:
        line = line.strip()
        if line:
            id_ko, *fields = line.split("\t")
            
            for i, v in enumerate(fields):
                id_field = header[i]
    
                try:
                    v = float(v)
                except ValueError:
                    v = v
                if v == "-":
                    v = None
                ko_to_data[id_ko][id_field] = v
        

# name_to_hmm = dict()
# missing_kos = set()
# for id_ko in tqdm(ko_to_data, desc="Loading HMMs", total=len(ko_to_data)):
#     ko_filepath = os.path.join(opts.kofam_database, "profiles", f"{id_ko}.hmm")
#     try:
#         with HMMFile(ko_filepath) as hmm_file:
#             for hmm in list(hmm_file):
#                 name_to_hmm[hmm.name.decode()] = hmm
#     except FileNotFoundError:
#         missing_kos.add(id_ko)
2
with SequenceFile(opts.proteins + ".gz", format="fasta", digital=True) as seqs_file:
    proteins = seqs_file.read_block()
# proteins

# {('SRR13615825__k127_151253_1', 'K07497'),
#  ('SRR13615825__k127_292915_2', 'K07497')}

In [35]:
from pyhmmer.easel import TextSequence, DigitalSequence, Alphabet
from Bio.SeqIO.FastaIO import SimpleFastaParser
# TextSequence(sequence=seq, name=str(i).encode()).digitize(hmm.alphabet)
# seq = TextSequence(sequence="MAAAAF", name="test".encode())
# seq.digitize("aa")

with gzip.open("./test.faa.gz", "rb") as f:
    for header, seq in SimpleFastaParser(f):
        id = header.split(" ")[0]
        break
id


'SRR13615825__k127_125066_1'

In [ ]:
df = pd.DataFrame(ko_to_data).T
df.loc[df["definition"].map(lambda x: "[EC:" not in x)]

In [163]:
import collections

def filter_threshold(
    hit, 
    threshold,
    score_type, 
    ):

    evalue = "{:0.5e}".format(hit.evalue)


    if score_type:
        if score_type == "domain":
            score = hit.best_domain.score
        else:
            score = hit.score
        if score >= threshold:
            return (threshold, score, evalue)
    else:
        return ("", score, evalue)
        
        
            

            
    
# #       gene name       KO      thrshld score   E-value "KO definition"
# Result = collections.namedtuple("Result", ["query", "ko", "bitscore", "evalue", "definition"])

hmms = [name_to_hmm["K00001"], name_to_hmm["K00002"]]
name_to_hmm2 = {
    "K00001":name_to_hmm["K00001"],
    "K00002":name_to_hmm["K00002"],
}

for id_ko, hmm in  tqdm(name_to_hmm.items(), desc="Performing HMMSearch", total=len(name_to_hmm)):
    if id_ko == "K01039":
    
        data = ko_to_data[id_ko]
        threshold = data["threshold"]
        score_type = data["score_type"]
        definition = data["definition"]
    
        
        for hits in hmmsearch(hmm, proteins, cpus=opts.n_jobs, E=opts.evalue):
            id_ko = hits.cog
            for hit in hits:
                if hit.included:
                    result = filter_threshold(hit, threshold, score_type)
                    if result:
                        threshold, score, evalue = result
                        print(hit.name.decode(), id_ko, threshold, score, evalue, definition, sep="\t")
                        
                    


Performing HMMSearch: 100%|███████████████████████████████████████████████████████████████████████████████████| 26162/26162 [00:00<00:00, 701338.24it/s]

SRR13615825__k127_281_1	K01039	171.83	382.19500732421875	5.60547e-115	glutaconate CoA-transferase, subunit A [EC:2.8.3.12]


In [166]:
# for hit in hits:
#     if hit.name.decode() == "SRR13615825__k127_281_1":
#         print(hit.name, hit.evalue, hit.best_domain.pvalue)
hit.

b'# 3 # 1298 # 1 # ID=4777_1;partial=11;start_type=Edge;rbs_motif=None;rbs_spacer=None;gc_cont=0.677'

In [ ]:

# with SequenceFile("data/seqs/938293.PRJEB85.HG003687.faa", digital=True) as seqs_file:
#     proteins = seqs_file.read_block()
    

# baseurl = "https://github.com/motu-tool/fetchMGs/raw/master/lib/{}.hmm"

# hmms = []
# for cog in cutoffs:
#     with urllib.request.urlopen(baseurl.format(cog)) as f:
#         with HMMFile(f) as hmm_file:
#             hmm = hmm_file.read()
#         cutoff = cutoffs[hmm.name.decode()]
#         hmm.cutoffs.trusted = (cutoff, cutoff)
#         hmms.append(hmm)

In [168]:
seq = proteins[0]
seq

In [152]:
df_meta_kofam.loc["K01039"]

threshold                                                  171.83
score_type                                                 domain
profile_type                                                  all
F-measure                                                0.996499
nseq                                                          960
nseq_used                                                     855
alen                                                         1496
mlen                                                          383
eff_nseq                                                     4.67
re/pos                                                      0.590
definition      glutaconate CoA-transferase, subunit A [EC:2.8...
Name: K01039, dtype: object

In [174]:
s = 0
with SequenceFile(opts.proteins, format="fasta", digital=True) as seqs_file:
    for block in seqs_file.read_block(sequences=10):
         s += len(k)
s

2142

In [177]:
k

In [2]:
import soothsayer as sy

Soothsayer Ecosystem
 * Soothsayer v2024.2.27
 * Soothsayer Utilities v2022.6.24
 * Compositional v2023.8.28
 * Hive NetworkX v2023.8.22
 * Ensemble NetworkX v2024.2.5
 
Auxillary Ecosystem
 * Python v3.9.18
 * Pandas v2.2.1
 * NumPy v1.26.4


In [23]:
ko_to_data, name_to_hmm = sy.io.read_object("/Users/jolespin/Databases/KOFAM/database.pkl")


In [24]:
# for id_ko, data in sy.pv(ko_to_data.items()):
#     definition = data["definition"]
#     ec = pd.NA
#     if "[EC:" in definition:
#         fields = definition.split("[EC:")
#         assert len(fields) == 2, fields
#         ec = fields[-1][:-1]
#         assert "." in ec
#     ko_to_data[id_ko]["ec"] = ec
# df = pd.DataFrame(ko_to_data).T


for id_ko, data in sy.pv(ko_to_data.items()):
    ec = ko_to_data[id_ko]["ec"]
    del ko_to_data[id_ko]["ec"]
    ko_to_data[id_ko]["enzyme_commission"] = ec
df = pd.DataFrame(ko_to_data).T
df.loc[df["enzyme_commission"].isnull()] = np.nan


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26598/26598 [00:00<00:00, 442944.88it/s]


In [39]:
df = pd.DataFrame(ko_to_data).T
# df
# .dropna(how="any",axis=0)

True

In [43]:
sy.io.write_object((ko_to_data, name_to_hmm), "/Users/jolespin/Databases/KOFAM/database.pkl")

In [48]:
ko_to_data__ec = dict()
name_to_hmm__ec = dict()
for id_ko, data in ko_to_data.items():
    ec = data["enzyme_commission"]
    if pd.notnull(ec):
        ko_to_data__ec[id_ko] = data
        name_to_hmm__ec[id_ko] = name_to_hmm[id_ko]

In [40]:
df.shape

(26598, 12)

In [49]:
len(ko_to_data__ec), len(name_to_hmm__ec)

(10544, 10544)

In [42]:
set(name_to_hmm.keys()) <= set(df.index)

True

In [54]:
sy.io.write_object((ko_to_data__ec, name_to_hmm__ec), "/Users/jolespin/Databases/KOFAM/database.enzymes.pkl")

In [47]:
from pandas import notnull
notnull(

In [53]:
df.loc[df["enzyme_commission"].notnull()]

,threshold,score_type,profile_type,F-measure,nseq,nseq_used,alen,mlen,eff_nseq,re/pos,definition,enzyme_commission
K00001,331.37,domain,all,0.307023,2853.0,2390.0,2242.0,502.0,14.92,0.59,alcohol dehydrogenase [EC:1.1.1.1],1.1.1.1
K00002,430.07,full,all,0.425739,2834.0,2714.0,7706.0,440.0,6.7,0.59,alcohol dehydrogenase (NADP+) [EC:1.1.1.2],1.1.1.2
K00003,284.43,domain,all,0.947269,7038.0,6099.0,3337.0,561.0,6.48,0.59,homoserine dehydrogenase [EC:1.1.1.3],1.1.1.3
K00004,368.53,domain,all,0.835152,1773.0,1506.0,1278.0,426.0,5.63,0.59,"(R,R)-butanediol dehydrogenase / meso-butanedi...",1.1.1.4 1.1.1.- 1.1.1.303
K00005,327.1,full,all,0.976968,1592.0,1182.0,793.0,372.0,2.16,0.59,glycerol dehydrogenase [EC:1.1.1.6],1.1.1.6
...,...,...,...,...,...,...,...,...,...,...,...,...
K27292,429.0,domain,all,1.0,126.0,123.0,862.0,556.0,2.2,0.589,3-aminoavenalumate diazotase [EC:6.7.1.2],6.7.1.2
K27305,None,None,None,None,2.0,2.0,3429.0,3195.0,0.54,0.594,polyketide synthase / 4-carboxy-3-alkylbut-2-e...,4.1.1.125
K27307,252.87,full,all,1.0,362.0,297.0,1073.0,423.0,1.59,0.59,acetyltransferase [EC:2.3.1.-],2.3.1.-
K27318,312.77,full,all,0.966168,1099.0,1066.0,5760.0,538.0,4.81,0.59,mitochondrial amidoxime-reducing component [EC...,1.7.-.-
